In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 96
NUM_CLASSES = 10
BATCH_SIZE = 32
EPOCHS_CLASSIFIER = 5
EPOCHS_FINE_TUNE = 5

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = tf.image.resize(X_train, (IMAGE_SIZE, IMAGE_SIZE)) / 255.0
X_test = tf.image.resize(X_test, (IMAGE_SIZE, IMAGE_SIZE)) / 255.0
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 45s 0us/step


In [5]:
# --- Transfer Learning Model Setup ---
base_model = DenseNet121(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:

# --- Phase 1: Train Custom Layers ---
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history1 = model.fit(X_train, y_train,
                     epochs=EPOCHS_CLASSIFIER,
                     batch_size=BATCH_SIZE,
                     validation_data=(X_test, y_test),
                     verbose=1)


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1634s 1s/step - accuracy: 0.6986 - loss: 0.8889 - val_accuracy: 0.7582 - val_loss: 0.6924
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1597s 1s/step - accuracy: 0.7958 - loss: 0.5728 - val_accuracy: 0.7918 - val_loss: 0.5963
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1645s 1s/step - accuracy: 0.8238 - loss: 0.4967 - val_accuracy: 0.7871 - val_loss: 0.6124
Epoch 4/5
1224/1563 ━━━━━━━━━━━━━━━━━━━━ 4:52 864ms/step - accuracy: 0.8480 - loss: 0.4303

In [ ]:

# --- Phase 2: Fine-Tuning ---
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history2 = model.fit(X_train, y_train,
                     epochs=EPOCHS_FINE_TUNE,
                     batch_size=BATCH_SIZE,
                     validation_data=(X_test, y_test),
                     verbose=1)



In [ ]:
# --- Accuracy Graph ---
history = {}
for key in history1.history.keys():
    history[key] = history1.history[key] + history2.history[key]

plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:

# --- Evaluation ---
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Accuracy: {accuracy*100:.2f}